<a href="https://colab.research.google.com/github/AnamHJ24/datascience-python-challenges/blob/main/notebooks/Day6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 6 - Ben & Jerry's
You are a Product Insights Analyst working with the **Ben & Jerry's** sales strategy team to
investigate seasonal sales patterns through comprehensive data analysis. The team wants to
understand how temperature variations and unique transaction characteristics impact ice
cream sales volume. Your goal is to perform detailed data cleaning and exploratory analysis
to uncover meaningful insights about seasonal sales performance.

In [12]:
# Import required libraries
import pandas as pd
import numpy as np

# Import data file
url = "https://raw.githubusercontent.com/AnamHJ24/datascience-python-challenges/refs/heads/main/Data/Day6.txt"
ice_cream_sales_data = pd.read_csv(url)
ice_cream_sales_data.head()

,sale_date,temperature,product_name,sales_volume,transaction_id
0,2024-07-05,62.0,Cherry Garcia,23,TX0001
1,2024-08-15,64.0,Chunky Monkey,26,TX0002
2,2024-09-25,66.0,Phish Food,29,TX0003
3,2024-10-05,68.0,Americone Dream,32,TX0004
4,2024-11-15,70.0,Chocolate Fudge Brownie,35,TX0005


# Question 1
Identify and remove any duplicate sales transactions from the dataset to ensure accurate analysis of
seasonal patterns.

# Solution

In [13]:
# Remove duplicates
ice_cream_sales_data_clean = ice_cream_sales_data.drop_duplicates()
ice_cream_sales_data_clean.head()

,sale_date,temperature,product_name,sales_volume,transaction_id
0,2024-07-05,62.0,Cherry Garcia,23,TX0001
1,2024-08-15,64.0,Chunky Monkey,26,TX0002
2,2024-09-25,66.0,Phish Food,29,TX0003
3,2024-10-05,68.0,Americone Dream,32,TX0004
4,2024-11-15,70.0,Chocolate Fudge Brownie,35,TX0005


## Question 2
Create a pivot table to summarize the total sales volume of ice cream products by month and
temperature range.<br />Use the following temperature bins where each bin excludes the upper bound
but includes the lower bound:<br />- Less than 60 degrees<br />- 60 to less than 70 degrees<br />-70
to less than 80 degrees<br />- 80 to less than 90 degrees<br />- 90 to less than 100 degrees<br />- 100
degrees or more

## Solution

In [24]:
# Convert required dataframes to datetime
ice_cream_sales_data_clean['sale_date'] = pd.to_datetime(ice_cream_sales_data_clean['sale_date'])
ice_cream_sales_data_clean = ice_cream_sales_data_clean.copy()
ice_cream_sales_data_clean.loc[:, 'sale_date'] = pd.to_datetime(ice_cream_sales_data_clean['sale_date'])

temp_bins = [0, 60, 70, 80, 90, 100, float('inf')]
labels = [
'Less than 60 degrees',
'60 to less than 70 degrees',
'70 to less than 80 degrees',
'80 to less than 90 degrees',
'90 to less than 100 degrees',
'100 degrees or more']
ice_cream_sales_data_clean['Temp range'] = pd.cut(ice_cream_sales_data_clean['temperature'],
                                                  bins = temp_bins,labels = labels,
                                                  right = False)
ice_cream_pivot_table = pd.pivot_table(ice_cream_sales_data_clean, values = 'sales_volume',
                                      index = [ice_cream_sales_data_clean['sale_date'].dt.month,'Temp range'],
                                      aggfunc = 'sum', observed=False)

print(ice_cream_pivot_table)

                                       sales_volume
sale_date Temp range                               
1         Less than 60 degrees                    0
          60 to less than 70 degrees            190
          70 to less than 80 degrees             41
          80 to less than 90 degrees            149
          90 to less than 100 degrees             0
...                                             ...
12        60 to less than 70 degrees             74
          70 to less than 80 degrees             90
          80 to less than 90 degrees            256
          90 to less than 100 degrees             0
          100 degrees or more                     0

[72 rows x 1 columns]


## Question 3
Can you detect any outliers in the monthly sales volume using the Inter Quartile Range (IQR) method?
A month is considered an outlier if falls below Q1 minus 1.5 times the IQR or above Q3 plus 1.5 times
the IQR.

## Solution

In [16]:
# Calculate total Sales aggregate
ice_cream_sales_data_clean['month'] = ice_cream_sales_data_clean['sale_date'].dt.month
ice_cream_sales_data_clean['year'] = ice_cream_sales_data_clean['sale_date'].dt.year
sales_agg = ice_cream_sales_data_clean.groupby(['year','month'])['sales_volume'].sum()
print(sales_agg)

year  month
2024  7        1414
      8         360
      9         375
      10        390
      11        405
      12        420
2025  1         435
      2         320
      3         335
      4         350
      5         365
      6        1324
Name: sales_volume, dtype: int64


In [20]:
# Calculate Q1, Q3 and IQR
q1 = sales_agg.quantile(0.25)
q3 = sales_agg.quantile(0.75)
iqr = q3 - q1
print("First Quartile(Q1):",q1)
print("Third Quartile(Q3):",q3)
print("Inter Quartile Range(IQR):",iqr)

First Quartile(Q1): 357.5
Third Quartile(Q3): 423.75
Inter Quartile Range(IQR): 66.25


In [21]:
# Find outliers
filtered = sales_agg[(sales_agg < (q1 - 1.5*iqr)) | (sales_agg > (q3 + 1.5*iqr))]
print("Months that are outliers:\n",filtered)

Months that are outliers:
 year  month
2024  7        1414
2025  6        1324
Name: sales_volume, dtype: int64
